In [1]:
"""Import relevant packages"""
from skimage import io
import numpy as np 
import os
from PIL import Image
import os
from GHOSTyolov5 import GHOSTyolov5
import time
import cv2
from skimage import io,filters,measure,feature,future,color,morphology
import  scipy.ndimage as ndi
import numpy as np
import matplotlib.pyplot as plt
from unet import  Unet
import pandas as pd
from scipy.ndimage import binary_hit_or_miss
import math
import tqdm


In [2]:
""""Loading deep learning models"""
def DeepLearning_Models():
    """Load YOLO and Unet models"""
    # Ensure the models are available in the current directory
    if not os.path.exists('GHOSTyolov5.py') or not os.path.exists('unet.py'):
        raise FileNotFoundError("YOLO and Unet model files are not found in the current directory.")    
    # GhostYOLOv5 model for object detection
    yolo = GHOSTyolov5()  # Initialize YOLO model
    # Unet model for instance segmentation
    unet = Unet()  # Initialize Unet model
    return yolo, unet
if __name__ == "__main__":
    yolo, unet = DeepLearning_Models()  # Load models
    print("Models loaded successfully.")
    # You can now use `yolo` and `unet` for further processing
    # For example, you can call yolo.detect_image(image) to perform detection

model_data\GHOSTyolov5_weights.pth model, and classes loaded.
model_data\unet_weights.pth model, and classes loaded.
Models loaded successfully.


In [17]:
""" Sort each file name"""
def sorted_file(Dir_path):
    filenames = [j for i , j in enumerate(os.listdir(Dir_path)) if j[-3:] == "jpg" or "png"]
    # filenames = os.listdir(Dir_path)  # List all files and folders in the current directory
    # sorted_filenames_ignorecase = sorted(filenames, key=str.lower)  # Sort
    sorted_filenames_ignorecase = sorted(filenames, key=str.lower) 
    # sorted_filenames_ignorecase = sorted(filenames)  # Sort in alphabetical order
    # Function to convert text to integer for natural sorting
    def atoi(text):
        return int(text) if text.isdigit() else text
    # Function to split text into natural keys for sorting
    def natural_keys(text): 
        return [atoi(c) for c in re.split('(\d+)', text)]
    import re
    sorted_filenames_natural = sorted(filenames, key=natural_keys)
    return sorted_filenames_natural

if __name__ == "__main__":
    pass
   
    
 
 

In [4]:
""""Obtain labels for each region and assign numbers"""
def seg_region(image):
    lab_image = color.rgb2lab(image)
    thresh=filters.threshold_otsu((lab_image[:,:,2]>filters.threshold_otsu(lab_image[:,:,2]))*lab_image[:,:,1])
    origin_mask=((lab_image[:,:,2]>filters.threshold_otsu(lab_image[:,:,2]))*lab_image[:,:,1])>thresh
    origin_mask=morphology.dilation(origin_mask,morphology.disk(15))
    origin_mask_fill=ndi.binary_fill_holes(origin_mask)
    contours, _ = cv2.findContours(origin_mask_fill.astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)   
    #Assuming we only care about the largest contour (i.e. the largest target object)
    #Find the contour with the largest area  
    max_contour = max(contours, key=cv2.contourArea) 
    # Draw the largest contour on a blank image 
    img = np.zeros((image.shape[0],image.shape[1]), dtype=np.uint8)
    x, y, w, h = cv2.boundingRect(max_contour) 
    # Draw the bounding box on the blank image
    from skimage.draw import line,rectangle
    img1 = np.zeros((image.shape[0],image.shape[1]), dtype=np.uint8)
    img2=morphology.erosion(img1.copy(),morphology.disk(1))
    rr1, cc1 = line(int((y+h/2)), x, int(y+h/2), x+w)  # Horizontal cutting line
    rr2, cc2 = line(y, int(x+w/3), y+h-1,int(x+w/3)) # Vertical 1 cutting line           
    rr3, cc3 = line(y, int(x+(w/3)*2), y+h-1,int(x+(w/3)*2))# Vertical 2 cutting line
    rr4, cc4 = rectangle((y, x), (y+h,x+w), shape=img.shape)
    img1[rr1, cc1] = 1
    img1[rr2, cc2] = 1
    img1[rr3, cc3] = 1
    img1=morphology.dilation(img1,morphology.disk(2))
    img2[rr4, cc4] = 1
    img3=np.where(img1>0,0,1)*img2
    return img3,x, y, w, h
if __name__ == "__main__": 
    pass
   

In [5]:
# Calculate the intersection to union ratio to obtain the maximum IOU
def IOU_NMS(box1, box2):
    # Get the coordinates of the top left and bottom right corners of box1
    x1min, y1min, x1max, y1max = box1[0], box1[1], box1[2], box1[3]
    # Calculate the area of box1
    s1 = (y1max - y1min + 1.) * (x1max - x1min + 1.)
    # Get the coordinates of the top left and bottom right corners of box2
    x2min, y2min, x2max, y2max = box2[0], box2[1], box2[2], box2[3]
    #Calculate the area of box2
    s2 = (y2max - y2min + 1.) * (x2max - x2min + 1.)
    # If the coordinates of the top left corner of box1 are greater than the bottom right corner, return 0
    xmin = np.maximum(x1min, x2min)
    ymin = np.maximum(y1min, y2min)
    xmax = np.minimum(x1max, x2max)
    ymax = np.minimum(y1max, y2max)
    # Calculate the height, width, and area of intersecting rectangular rows
    inter_h = np.maximum(ymax - ymin + 1., 0.)
    inter_w = np.maximum(xmax - xmin + 1., 0.)
    intersection = inter_h * inter_w
    # Calculate the combined area
    union = s1 + s2 - intersection
    # Calculate the intersection to union ratio
    iou = intersection / union
    return iou
if __name__ == "__main__":
    pass

In [6]:
""""Image matching """
#We need a function here to remove duplicate boxes from a single image
#The logic can be like this: match the last layer of the previous level and then match the next layer
def computer_coord(box):
    top, left, bottom, right = box
    top     = max(0, np.floor(top).astype('int32'))
    left    = max(0, np.floor(left).astype('int32'))
    coord=(np.int32((bottom-top)/2+top),np.int32((left+(right-left)/2)))
    return coord
def next_fps_coord( BOX_IMAGE_1, BOX_IMAGE_2):
    COORD=[] 
    CONNECT_BOX =[]
    BEGAIN_COORD=[]
    NEXT=[]
    for i1 ,j1 in enumerate(BOX_IMAGE_1):
        box1=BOX_IMAGE_1[i1]
        min_iou= 0.5
        next_coord=()
        next_box=[]
        for i2, j2 in enumerate(BOX_IMAGE_2): #Calculate the maximum intersection IOU with the next frame image
            box2=BOX_IMAGE_2[i2]
            IOU=IOU_NMS(box1,box2)
            if IOU > min_iou:
                # print(IOU)
                min_iou = IOU
                next_coord= computer_coord(box2)
                next_box = box2
        if next_box ==[]:
            next_coord= computer_coord(box1)
            next_box = box1
        COORD.append(next_coord)
        CONNECT_BOX.append(next_box)
    return COORD,CONNECT_BOX
if __name__ == "__main__":
    pass

In [7]:
"""External frame characteristics"""
def rectangle_traits(Region,mm_pix_radio):
    """Calculate the width, length, area, perimeter, and aspect ratio of rectangles in the given regions."""
    germinate_rectangle_width=[]
    germinate_rectangle_length=[]
    germinate_rectangle_area=[]
    germinate_rectangle_perimeter=[]
    germinate_rectangle_aspect_radio=[]
    for i , j in enumerate(Region):
        width=[]
        length=[]
        area=[]
        perimeter=[]
        aspect_radio =[]
        for i1,j1 in enumerate(Region[i]):
            top, left, bottom, right = Region[i][i1]
            width.append((right-left)*mm_pix_radio)
            length.append((bottom-top)*mm_pix_radio)
            area.append(((right-left)*(bottom-top))*mm_pix_radio)
            perimeter.append(((right-left)+(bottom-top))*2*mm_pix_radio)
            aspect_radio.append((right-left)/(bottom-top))
            # print("width:",width,"length:",length,"area:",area,"perimeter:",perimeter,"aspect_radio:",aspect_radio)     
        germinate_rectangle_width.append(width)
        germinate_rectangle_length.append(length)
        germinate_rectangle_area.append(area)
        germinate_rectangle_perimeter.append(perimeter)
        germinate_rectangle_aspect_radio.append(aspect_radio)
    return  germinate_rectangle_width, germinate_rectangle_length,\
            germinate_rectangle_area,germinate_rectangle_perimeter,\
            germinate_rectangle_aspect_radio
if __name__ == "__main__":
    pass
              

In [8]:
"""Reconstruction of growth trajectory"""
def track_extraction(Track,mm_pix_radio,time_interval): # -> mm_pix_radio: Conversion of actual distance of culture dish to pixels 
    Linear_distance =[]
    """Calculate the linear distance, speed, and mean square displacement of the growth trajectory."""
    Linear_speed = []
    Linear_MSD=[]
    for i , j in enumerate(Track):
        DISTANCE=[]
        SPEED=[]
        MSD=[]
        label=Track[i]
        if (() in label) == True:
            label.remove(())
        if len(label) == 1:
                DISTANCE = [0]
                SPEED =  [0]
                MSD =  [0]
        if len(label) != 1:
            for x ,y in enumerate(label):
                if x == len(label)-1 :
                    continue
                coord_1=label[x]
                coord_2=label[x+1]
                DIS=(pow(((coord_2[1]-coord_1[1])**2+(coord_2[0]-coord_1[0])**2),0.5))*mm_pix_radio
                M_dis =((coord_2[1]-label[0][1])**2+(coord_2[0]-label[0][0])**2)*mm_pix_radio
                #  速度
                SPE=DIS/time_interval
                if DISTANCE == []:
                    DISTANCE.append(DIS)
                if DISTANCE !=[]:
                    DISTANCE.append(DIS+DISTANCE[-1])
                SPEED.append(SPE)
                MSD.append(M_dis)
        Linear_distance.append(DISTANCE)
        Linear_speed.append(SPEED)
        Linear_MSD.append(np.mean(MSD))
    return Linear_distance,Linear_speed,Linear_MSD
def track_traits(Region,mm_pix_radio,time_interval):
    germinate_centroid =[]
    for i , j in enumerate(Region):
        centroid=[]
        for i1,j1 in enumerate(Region[i]):
            top, left, bottom, right = Region[i][i1]
            centroid.append([np.int32((left+(right-left)/2)),np.int32((bottom-top)/2+top)])
        germinate_centroid.append(centroid)
    Linear_distance,Linear_speed,Linear_MSD=track_extraction(germinate_centroid,mm_pix_radio,time_interval)
    return Linear_distance,Linear_speed,Linear_MSD
if __name__ == "__main__": 
    pass

In [9]:
"""External frame area change rate"""
def track_area(Track_Area,mm_pix_radio,time_interval):
    """Calculate the growth rate of the area in each frame."""
    Growth_Rate=[]
    for i , j in enumerate(Track_Area):
        Growth=[]
        label=Track_Area[i]
        if (() in label) == True:
            label.remove(())
        if len(label) == 1:
                Growth=[0]
        if len(label) != 1:
            for x ,y in enumerate(label):
                if x == len(label)-1 :
                    continue
                area_1=label[x]
                area_2=label[x+1]
                DIS=(area_2[0]-area_1[0])*mm_pix_radio
                if DIS <= 0 :
                    DIS = 0
                Growth.append(DIS/time_interval)
        Growth_Rate.append(Growth)
    return Growth_Rate
def Area_change(Region,mm_pix_radio,time_interval):
    germinate_rectangle_area=[]
    for i , j in enumerate(Region):
        area=[]
        for i1,j1 in enumerate(Region[i]):
            top, left, bottom, right = Region[i][i1]
            area.append([(right-left)*(bottom-top)])
        germinate_rectangle_area.append(area)
    rectangle_area_speed=track_area( germinate_rectangle_area,mm_pix_radio,time_interval)
    return rectangle_area_speed
if __name__ == "__main__":
    pass

In [10]:
"""Keep the maximum area mask"""
def segmation_rol(mask):
    """Segment the mask to keep only the largest area."""
    # This function labels connected components in the mask and retains only the largest one.
    from skimage import io ,measure
    label_img=measure.label(mask)
    properties=measure.regionprops(label_img)
    a=[]
    for prop in properties:
        a.append(prop.area)
    valid_label = set()
    for prop in properties:
        if prop.area == max(a):
            valid_label.add(prop.label)
    big_mask= np.in1d(label_img, list(valid_label)).reshape(label_img.shape)
    return big_mask
if __name__ == "__main__":
    pass

In [11]:
"""endpoint detection"""
def find_end_points(skel):
    """Detect end points of a skeleton"""
    # Four possible matrix representation
    struct1, origin1 = np.array([
            [0, 0, 0],
            [0, 1, 0],
        ]), (0, 0)
    struct2, origin2 = np.array([
            [0, 0],
            [0, 1],
            [0, 0],
        ]), (0, 0)
    struct3, origin3 = np.array([
            [0, 1, 0],
            [0, 0, 0],
        ]), (-1, 0)
    struct4, origin4 = np.array([
            [0, 0],
            [1, 0],
            [0, 0],
        ]), (0, -1)
    
    # Match end point structures with the skeleton
    ret = None
    for i in range(1, 5):
        struct, origin = locals()['struct%d' % (i)], locals()['origin%d' % (i)]
        if ret is None:
            ret = binary_hit_or_miss(skel, structure1=struct, origin1=origin)
        else:
            ret = np.logical_or(ret, binary_hit_or_miss(skel, structure1=struct, origin1=origin))
    return np.nonzero(ret)[::-1]
peak_counter=0
#
def find_end_image(img,sk1,font=cv2.FONT_HERSHEY_SIMPLEX, font_size=3, font_thickness=2):
    ret=find_end_points(sk1)
    black_im=np.zeros((img.shape[0],img.shape[1]))
    peak_counter = 0
    for i in range(len(ret[0])):
        black_im[ret[1][i],ret[0][i]]=1
        text = str(peak_counter + 1)
#         cv2.line(img, (int(cx), int(cy)), (int(ret[0][i]), int(ret[1][i])), (255, 0, 0), 10)
        (text_w, text_h), baseline = cv2.getTextSize(text, font, font_size, font_thickness)
        # compute the text offsets (x, y)
        tx = int(round(ret[0][i] - (text_w / 2.0)))
        ty = int(round(ret[1][i] + (text_h / 2.0)))
        # render the text
        cv2.circle(img, (ret[0][i], ret[1][i]), 5, (255, 255, 0), -1)
        cv2.circle(img, (ret[0][i],ret[1][i]), 1, (255, 0, 0), 1)
        cv2.putText(img, text, (tx, ty), font, font_size, (255, 0, 0), font_thickness)
        peak_counter += 1
    return black_im,img,ret

if __name__ == "__main__":
    pass

In [12]:
"""get traits of dynamics"""

def get_begain_root_tip_point(ret1,ret0): 
    
    #
    max_dis = 10000 

    for i in range(len(ret1[0])):
        for i1 in range(len(ret0[0])):
      
            DIS=(pow(((ret1[0][i]-ret0[0][i1])**2+(ret1[1][i]-ret0[1][i1])**2),0.5))
            if DIS <= max_dis:
                max_dis = DIS
                near_point = (ret1[0][i],ret1[1][i])
             
    return near_point, DIS

def get_end_root_tip_point(ret1,ret0): 
    point,DIS = get_begain_root_tip_point(ret1,ret0)
    max_dis = 0
    for i in range(len(ret1[0])):
        DIS=(pow(((ret1[0][i]-point[0])**2+(ret1[1][i]-point[1])**2),0.5))
        if DIS >= max_dis: 
            max_dis = DIS
          
            root_tip_point = (ret1[0][i],ret1[1][i])
    return root_tip_point,point,DIS

def coord_transfer(coord,top,left): 
    coord_origin = (coord[1]+top,coord[0]+left)
    return coord_origin

def average_no_zero(numbers):
    filtered_numbers = [x for x in numbers if x != 0]
 
    if filtered_numbers:  
        average = sum(filtered_numbers) / len(filtered_numbers)
    else:
        average = 0  
    return average


def seed_root_buds_traits (sorted_filenames_natural,mm_pix_radio,Region,dir_path):
   
    Seed_length=[]
    Seed_width = []
    Seed_area = []
    Seed_perimeter =[]
    Seed_eccentricity =[]
    Seed_roundness =[]
    Seed_aspect_radio =[]
 
    Root_area=[]
    Root_length =[]
    Root_diameter = []
    Root_permiter =[]
    White_time = []
    White_radio = []
    Shoot_length =[]
    Shoot_area =[]
    Shoot_diameter =[]
    Shoot_permiter = []
    Root_Coord = []
    Germination_rate= [] 
    Curling_degree = []
  
    for i, j in enumerate(Region):
        save =  True
        Track = True
        R_skl_mask=[]
        R_area=[]
        R_length =[]
        R_diameter = []
        R_perimeter =[]
        W_time = []  
        W_radio = [] 
        G_num = []  
     
        S_length =[]
        S_area =[]
        S_diameter =[] 
        S_permiter =[] 
       
        Coord = []
        seed_mask = [] 
        Curling = []
        num = 1
   
        for i1,j1 in enumerate(Region[i]):
            img = Image.open(os.path.join(dir_path,sorted_filenames_natural[i1]))
            top, left, bottom, right = Region[i][i1]
            top     = max(0, np.floor(top).astype('int32'))
            left    = max(0, np.floor(left).astype('int32'))
            bottom  = min(img.size[1], np.floor(bottom).astype('int32'))
            right   = min(img.size[0], np.floor(right).astype('int32'))
            crop_image = img.crop([left, top, right, bottom])
            mask,image= unet.detect_image(crop_image, count=False)
            # num = 1
            if save == True:
                dir_save_path = "./Region_Crop"
                if not os.path.exists(dir_save_path):
                    os.makedirs(dir_save_path)
                Region_dir = os.path.join(dir_save_path,f"Seed_single{i+1}")
                if not  os.path.exists(Region_dir):
                    os.makedirs(Region_dir)
                crop_image.save(os.path.join(Region_dir, "crop_" + str(i1+1) + ".png"), quality=95, subsampling=0)
            if i1 == 0 : 
                mask = segmation_rol(np.where(mask ==3,1,0))
                seed_mask.append(mask) 
                region=measure.regionprops(measure.label(seed_mask[0]))
                Seed_length.append(region[0].axis_major_length*mm_pix_radio)
                Seed_width.append(region[0].axis_minor_length*mm_pix_radio)
                Seed_area.append(region[0].area*mm_pix_radio)
                Seed_perimeter.append(region[0].perimeter*mm_pix_radio)
                Seed_eccentricity.append(region[0].eccentricity)
                Seed_roundness.append((region[0].area*4*math.pi)/pow(region[0].perimeter,2))
                Seed_aspect_radio.append((region[0].axis_major_length/region[0].axis_minor_length))
           
            root_mask = morphology.closing(morphology.opening(np.where(mask ==2,1,0),morphology.disk(1)),morphology.disk(1))
            root_mask = morphology.remove_small_objects(root_mask.astype(np.bool8),50)  
            buds_mask = morphology.closing(morphology.opening(np.where(mask ==1,1,0),morphology.disk(1)),morphology.disk(1))
            buds_mask = morphology.remove_small_objects(buds_mask.astype(np.bool8),40)
        
            x=[region.area for region in (measure.regionprops(measure.label(root_mask)))]
            y=[region.perimeter for region in (measure.regionprops(measure.label(buds_mask)))]
            z=[ region.area for region in (measure.regionprops(measure.label(buds_mask)))]
            w=[region.perimeter for region in (measure.regionprops(measure.label(buds_mask)))]
            roots_skl=morphology.skeletonize(root_mask) 
            seed_skl=morphology.skeletonize(segmation_rol(np.where(mask ==3,1,0))) 
            if measure.perimeter(roots_skl)*mm_pix_radio >= (Seed_length[0])*0.5: 
                G_num.append(1)  
            if measure.perimeter(roots_skl)*mm_pix_radio < (Seed_length[0])*0.5: 
                if  G_num != [] and G_num[-1] == 1:
                    G_num.append(1) 
                else: 
                    G_num.append(0)
            if np.sum (x)>= 50:
          
                if Track == True:
                        ret1=find_end_points(roots_skl)
                        ret0=find_end_points(seed_skl)   
                        ret1=([ret1[0][i]+left for i in range(len(ret1[0]))],[ret1[1][i]+top for i in range(len(ret1[0]))])
                        ret0=([ret0[0][i]+left for i in range(len(ret0[0]))],[ret0[1][i]+top for i in range(len(ret0[0]))])
                        if ret0 == ([],[]):
                           if Coord == [] or Coord[-1] == (0,0):
                               ret0=([seed_skl.shape[1]//2],[seed_skl.shape[0]//2])
                           else:
                               ret0=([Coord[-1][0][0]],[Coord[-1][0][1]])
                      
                        if ret1 == ([],[]):
                           ret1=([Coord[-1][1][0]],[Coord[-1][1][1]])
                        root_tip_point,near_point,DIS = get_end_root_tip_point(ret1,ret0) 
                        if root_tip_point == near_point and Coord[-1] != (0,0):
                            root_tip_point =  Coord[-1][1]
                Coord.append([near_point,root_tip_point])  
                R_area.append(np.sum(x)*mm_pix_radio)
                rl=measure.perimeter(roots_skl)*mm_pix_radio
                Curling.append(((pow(((near_point[0]-root_tip_point[0])**2+(near_point[1]-root_tip_point[1])**2),0.5))*mm_pix_radio)/rl)
                R_length.append(rl)  
                if num == 1:
                    W_time.append(i1 +1)
                    num = num+1
                W_radio.append(1)
                R_perimeter.append(np.sum(y)*mm_pix_radio)
                R_diameter.append((np.sum(x)*mm_pix_radio)/rl)
            if np.sum(x) < 50:
                R_length.append(0)
                R_area.append(0)
                R_perimeter.append(0)
                R_diameter.append(0) 
                Coord.append((0,0))  
                Curling.append(0)  
                if  R_area !=[] and  R_area[-1] >= 50*mm_pix_radio:
                   
                    W_radio.append(W_radio[-1]) 
                    
                else :
                    W_radio.append(0)
                   
                        
            if np.sum(z)>= 40:
                    S_area.append(np.sum(z)*mm_pix_radio)
                    buds_skl = morphology.skeletonize(buds_mask)
                    bl=measure.perimeter(buds_skl)*mm_pix_radio
                    S_length.append(bl)
                    S_permiter.append(np.sum(w)*mm_pix_radio)
                    S_diameter.append((np.sum(z)*mm_pix_radio)/bl)
            if np.sum(z)< 40:
                if len(S_area)!=0 and S_area[-1] >= 40*mm_pix_radio:
                    S_area.append(S_area[-1])
                    S_length.append(S_length[-1])
                    S_permiter.append(S_permiter[-1])
                    S_diameter.append(S_diameter[-1])
                else:
                    S_area.append(0)
                    S_length.append(0)
                    S_permiter.append(0)
                    S_diameter.append(0)
        if W_time == []:
            W_time.append(0)
        Root_area.append(R_area) 
        Root_length.append(R_length) 
        Root_diameter.append(R_diameter) 
        Root_permiter.append(R_perimeter) 
        Shoot_length.append(S_length) 
        Shoot_area.append(S_area) 
        Shoot_diameter.append(S_diameter) 
        Shoot_permiter.append(S_permiter) 
        White_time.append(W_time[0]) 
        White_radio.append(W_radio) 
        Root_Coord.append(Coord) 
        Germination_rate.append(G_num) 
        Curling_degree.append( Curling) 

    return  Seed_length, Seed_width, Seed_area ,Seed_perimeter , Seed_eccentricity ,Seed_roundness ,Seed_aspect_radio ,Root_area,\
            Root_length, Root_diameter,Root_permiter,Shoot_length,Shoot_area,Shoot_diameter, White_time, White_radio, Root_Coord,\
            Germination_rate, Curling_degree,Shoot_permiter
    



In [13]:
"""Based on IOU correlation, calculate the BOX for each region"""
def get_box(dir_path ,sorted_filenames_natural):
    SAVE_detect_image = False
    """Detect images in the specified directory and return the bounding boxes and germination rates."""
    region_path = os.path.join(dir_path,sorted_filenames_natural[0])
    img = io.imread(region_path)
    region_mask,x, y, w, h = seg_region(img)
    # You can edit coold_N according to the actual area
    coord1= [x,y,x+w/3,y+h/2]
    coord2= [x+w*1/3,y,x+w*2/3,y+h/2]
    coord3= [x+w*2/3,y,x+w,y+h/2]
    coord4= [x,y+h/2,x+w*1/3,y+h]
    coord5= [x+w*1/3,y+h*1/2,x+w*2/3,y+h]
    coord6= [x+(w*2/3),y+h*1/2,x+w,y+h]
    region_coord = [coord1,coord2,coord3, coord4, coord5, coord6]
    if SAVE_detect_image == True:  
        BASE_DIR=os.getcwd()
        SAVE_IMAGE_PATH = os.path.join(BASE_DIR,"SINGLE_DETEXT_IMAGE")
        if os.path.exists(SAVE_IMAGE_PATH) == False:
            os.mkdir(SAVE_IMAGE_PATH)
    Class = "dir"
    if Class == "dir":
        TOTAL_BOX = []
        GERMINATION_RATE =[] 
        for i , File in enumerate(sorted_filenames_natural): 
            FILE_NAME=sorted_filenames_natural[i][:-3]
            image = Image.open(os.path.join(dir_path,sorted_filenames_natural[i]))

            r_image,BOX = yolo.detect_image(image, region_coord, seed_num=9,crop = False, count=False)  
            
            #Seed_num is based on the actual number of seeds placed
            TOTAL_BOX.append(BOX)
            # GERMINATION_RATE.append(radio)
            if SAVE_detect_image == True:  
                io.imsave(os.path.join(SAVE_IMAGE_PATH,sorted_filenames_natural[i]),np.array(r_image))
        COORD=[[computer_coord(TOTAL_BOX[0][i])] for i , j in enumerate(TOTAL_BOX[0])]  
        BOX=[[TOTAL_BOX[0][i]] for i , j in enumerate(TOTAL_BOX[0])]
        # COORD=[computer_coord(TOTAL_BOX[0])]
        for i1 , j1 in enumerate(TOTAL_BOX):
            if i1 == len(TOTAL_BOX)-1:
                continue
            LAST_CONNECT =[BOX[i2][-1] for i2 , j2 in enumerate(BOX)]
            # if num == 0:
            next_coord,next_box=next_fps_coord(LAST_CONNECT,TOTAL_BOX[i1+1])
           
            for i3, j3 in enumerate(next_coord):
                COORD[i3].append(next_coord[i3])
            for i4, j4 in enumerate(next_box):
                BOX[i4].append(next_box[i4])
    return BOX,region_coord
def get_six_region(BOX,region_coord):
    """Divide the bounding boxes into six regions based on predefined coordinates."""
    """This function takes a list of bounding boxes and divides them into six regions based on the"""
    Region_one = []
    Region_two = []
    Region_three = []
    Region_four = []
    Region_five = []
    Region_six = []
    for i , j in enumerate(BOX):
        top, left, bottom, right = BOX[i][0]
        centroid=[np.int32((left+(right-left)/2)),np.int32((bottom-top)/2+top)]     
        # Check which region the centroid of the bounding box falls into
        if region_coord[0][0] <centroid[0] < region_coord[0][2] and region_coord[0][1] < centroid[1] <region_coord[0][3] :   
                Region_one.append(BOX[i])
        # The first region
               
        elif region_coord[1][0] <centroid[0] < region_coord[1][2] and region_coord[1][1] < centroid[1] <region_coord[1][3] :     
                Region_two.append(BOX[i])
        # The second region

        elif region_coord[2][0] <centroid[0] < region_coord[2][2] and region_coord[2][1] < centroid[1] <region_coord[2][3] :     
                Region_three.append(BOX[i])
        # The third region
               
        elif region_coord[3][0] <centroid[0] < region_coord[3][2] and region_coord[3][1] < centroid[1] <region_coord[3][3] :  
                Region_four.append(BOX[i])
               
        # The fourth  region
        elif region_coord[4][0] <centroid[0] < region_coord[4][2] and region_coord[4][1] < centroid[1] <region_coord[4][3] :  
                Region_five.append(BOX[i])
                
        # The fifth region
        elif region_coord[5][0] <centroid[0] < region_coord[5][2] and region_coord[5][1] < centroid[1] <region_coord[5][3] :     
                Region_six.append(BOX[i])
        # The sixth region
    # Return the six regions as lists of bounding boxes
               
    return Region_one,Region_two,Region_three,Region_four,Region_five,Region_six
if __name__ == "__main__":
    pass
#     dir_path = "Demo_data"
#     mm_pix_radio = 0.2165405645815346785
#     sorted_filenames_natural = sorted_file(dir_path)
#     BOX,region_coord = get_box (dir_path ,sorted_filenames_natural)
#     Region_one,Region_two,Region_three,Region_four,Region_five,Region_six =get_six_region(BOX,region_coord)
  

    

In [ ]:
if __name__ == "__main__":
  
    Dir = r"Demo_Datas"
    for i , j in enumerate(tqdm.tqdm(os.listdir(Dir))):
      
        dir_path = os.path.join(Dir,j)
        
        sorted_filenames_natural = sorted_file(dir_path)
        BOX,region_coord=get_box(dir_path ,sorted_filenames_natural)
        mm_pix_radio = 0.2165405645815346785 
      
        time_interval = 1
        Root_Area_threshold = 50
        Root_skl_threshold = 20
        Buds_area_threshold = 40
        Region_one,Region_two,Region_three,Region_four,Region_five,Region_six = get_six_region(BOX,region_coord) 
       
        germinate_rectangle_width_1, germinate_rectangle_length_1,\
                    germinate_rectangle_area_1,germinate_rectangle_perimeter_1,\
                    germinate_rectangle_aspect_radio_1=rectangle_traits(Region_one,mm_pix_radio)
        germinate_rectangle_width_2, germinate_rectangle_length_2,\
                    germinate_rectangle_area_2,germinate_rectangle_perimeter_2,\
                    germinate_rectangle_aspect_radio_2=rectangle_traits(Region_two,mm_pix_radio)
        germinate_rectangle_width_3, germinate_rectangle_length_3,\
                    germinate_rectangle_area_3,germinate_rectangle_perimeter_3,\
                    germinate_rectangle_aspect_radio_3=rectangle_traits(Region_three,mm_pix_radio)
        germinate_rectangle_width_4, germinate_rectangle_length_4,\
                    germinate_rectangle_area_4,germinate_rectangle_perimeter_4,\
                    germinate_rectangle_aspect_radio_4=rectangle_traits(Region_four,mm_pix_radio)
        germinate_rectangle_width_5, germinate_rectangle_length_5,\
                    germinate_rectangle_area_5,germinate_rectangle_perimeter_5,\
                    germinate_rectangle_aspect_radio_5=rectangle_traits(Region_five,mm_pix_radio)
        germinate_rectangle_width_6, germinate_rectangle_length_6,\
                    germinate_rectangle_area_6,germinate_rectangle_perimeter_6,\
                    germinate_rectangle_aspect_radio_6=rectangle_traits(Region_six,mm_pix_radio)

        Linear_distance1,Linear_speed1,Linear_MSD1 = track_traits(Region_one,mm_pix_radio,time_interval)
        Linear_distance2,Linear_speed2,Linear_MSD2 = track_traits(Region_two,mm_pix_radio,time_interval)
        Linear_distance3,Linear_speed3,Linear_MSD3 = track_traits(Region_three,mm_pix_radio,time_interval)
        Linear_distance4,Linear_speed4,Linear_MSD4 = track_traits(Region_four,mm_pix_radio,time_interval)
        Linear_distance5,Linear_speed5,Linear_MSD5 = track_traits(Region_five,mm_pix_radio,time_interval)
        Linear_distance6,Linear_speed6,Linear_MSD6 = track_traits(Region_six,mm_pix_radio,time_interval)

        germinate_rectangle_speed1 = Area_change(Region_one,mm_pix_radio,time_interval)
        germinate_rectangle_speed2 = Area_change(Region_two,mm_pix_radio,time_interval)
        germinate_rectangle_speed3 = Area_change(Region_three,mm_pix_radio,time_interval)
        germinate_rectangle_speed4 = Area_change(Region_four,mm_pix_radio,time_interval)
        germinate_rectangle_speed5 = Area_change(Region_five,mm_pix_radio,time_interval)
        germinate_rectangle_speed6 = Area_change(Region_six,mm_pix_radio,time_interval)

        Seed_length1, Seed_width1, Seed_area1 ,Seed_perimeter1 , Seed_eccentricity1 ,Seed_roundness1 ,Seed_aspect_radio1 ,Root_area1,\
        Root_length1, Root_diameter1,Root_permiter1,Shoot_length1,Shoot_area1,Shoot_diameter1, White_time1, White_radio1, Root_Coord1,\
        Germination_rate1, Curling_degree1,Shoot_permiter1 = seed_root_buds_traits (sorted_filenames_natural,mm_pix_radio,Region_one,dir_path)

        Seed_length2, Seed_width2, Seed_area2 ,Seed_perimeter2 , Seed_eccentricity2,Seed_roundness2,Seed_aspect_radio2,Root_area2,\
        Root_length2, Root_diameter2,Root_permiter2,Shoot_length2,Shoot_area2,Shoot_diameter2, White_time2, White_radio2, Root_Coord2,\
        Germination_rate2, Curling_degree2,Shoot_permiter2 = seed_root_buds_traits (sorted_filenames_natural,mm_pix_radio,Region_two,dir_path)

        Seed_length3, Seed_width3, Seed_area3,Seed_perimeter3 , Seed_eccentricity3,Seed_roundness3,Seed_aspect_radio3,Root_area3,\
        Root_length3, Root_diameter3,Root_permiter3,Shoot_length3,Shoot_area3,Shoot_diameter3, White_time3, White_radio3, Root_Coord3,\
        Germination_rate3, Curling_degree3 ,Shoot_permiter3= seed_root_buds_traits (sorted_filenames_natural,mm_pix_radio,Region_three,dir_path)

        Seed_length4, Seed_width4, Seed_area4 ,Seed_perimeter4 , Seed_eccentricity4,Seed_roundness4,Seed_aspect_radio4,Root_area4,\
        Root_length4, Root_diameter4,Root_permiter4,Shoot_length4,Shoot_area4,Shoot_diameter4, White_time4, White_radio4, Root_Coord4,\
        Germination_rate4, Curling_degree4,Shoot_permiter4 = seed_root_buds_traits (sorted_filenames_natural,mm_pix_radio,Region_four,dir_path)

        Seed_length5, Seed_width5, Seed_area5,Seed_perimeter5, Seed_eccentricity5,Seed_roundness5,Seed_aspect_radio5,Root_area5,\
        Root_length5, Root_diametedr5,Root_permiter5,Shoot_length5,Shoot_area5,Shoot_diameter5, White_time5, White_radio5, Root_Coord5,\
        Germination_rate5, Curling_degree5,Shoot_permiter5 = seed_root_buds_traits (sorted_filenames_natural,mm_pix_radio,Region_five,dir_path)

        Seed_length6, Seed_width6, Seed_area6,Seed_perimeter6, Seed_eccentricity6,Seed_roundness6,Seed_aspect_radio6,Root_area6,\
        Root_length6, Root_diameter6,Root_permiter6,Shoot_length6,Shoot_area6,Shoot_diameter6, White_time6, White_radio6, Root_Coord6,\
        Germination_rate6, Curling_degree6,Shoot_permiter6 = seed_root_buds_traits(sorted_filenames_natural,mm_pix_radio,Region_six,dir_path)

        x =[f"{i+1}(hours)" for i in range(len(sorted_filenames_natural))]

    x1 =[f"{i+2}(hours)" for i in range(len(sorted_filenames_natural)-1)]

    y1 = [f"Region_one_seed{i+1}(mm)" for i in range(9)]+[f"Region_two_seed{i+1}(mm)" for i in range(9)]+\
        [f"Region_three_seed{i+1}(mm)" for i in range(9)]+[f"Region_four_seed{i+1}(mm)" for i in range(9)]+\
        [f"Region_five_seed{i+1}(mm)" for i in range(9)]+[f"Region_six_seed{i+1}(mm)" for i in range(9)]

    y2 = [f"Region_one_seed{i+1}(mm²)" for i in range(9)]+[f"Region_two_seed{i+1}(mm²)" for i in range(9)]+\
        [f"Region_three_seed{i+1}(mm²)" for i in range(9)]+[f"Region_four_seed{i+1}(mm²)" for i in range(9)]+\
        [f"Region_five_seed{i+1}(mm²)" for i in range(9)]+[f"Region_six_seed{i+1}(mm²)" for i in range(9)]

    y3 = [f"Region_one_seed{i+1}(mm/h)" for i in range(9)]+[f"Region_two_seed{i+1}(mm/h)" for i in range(9)]+\
        [f"Region_three_seed{i+1}(mm/h)" for i in range(9)]+[f"Region_four_seed{i+1}(mm/h)" for i in range(9)]+\
        [f"Region_five_seed{i+1}(mm/h)" for i in range(9)]+[f"Region_six_seed{i+1}(mm/h)" for i in range(9)]

    y4 = [f"Region_one_seed{i+1}(0-1)" for i in range(9)]+[f"Region_two_seed{i+1}(0-1)" for i in range(9)]+\
        [f"Region_three_seed{i+1}(0-1)" for i in range(9)]+[f"Region_four_seed{i+1}(0-1)" for i in range(9)]+\
        [f"Region_five_seed{i+1}(0-1)" for i in range(9)]+[f"Region_six_seed{i+1}(0-1)" for i in range(9)]


    Save_dir = "./"+os.path.basename(dir_path)+"traits"
    if os.path.exists(Save_dir) == False:
        os.mkdir(Save_dir)

    
    df_seed_traits=pd.DataFrame([[np.mean(Seed_length1),np.mean(Seed_length2),np.mean(Seed_length3),np.mean(Seed_length4),np.mean(Seed_length5),np.mean(Seed_length6)],
                [np.mean(Seed_width1),np.mean(Seed_width2),np.mean(Seed_width3),np.mean(Seed_width4),np.mean(Seed_width5),np.mean(Seed_width6)],
                [np.mean(Seed_area1),np.mean(Seed_area2),np.mean(Seed_area3),np.mean(Seed_area4),np.mean(Seed_area5),np.mean(Seed_area6)],
                [np.mean(Seed_perimeter1),np.mean(Seed_perimeter2),np.mean(Seed_perimeter3),np.mean(Seed_perimeter4),np.mean(Seed_perimeter5),np.mean(Seed_perimeter6)],
                [np.mean(Seed_eccentricity1),np.mean(Seed_eccentricity2),np.mean(Seed_eccentricity3),np.mean(Seed_eccentricity4),np.mean(Seed_eccentricity5),np.mean(Seed_eccentricity6)],
                [np.mean(Seed_roundness1),np.mean(Seed_roundness2),np.mean(Seed_roundness3),np.mean(Seed_roundness4),np.mean(Seed_roundness5),np.mean(Seed_roundness6)],
                [np.mean(Seed_aspect_radio1),np.mean(Seed_aspect_radio2),np.mean(Seed_aspect_radio3),np.mean(Seed_aspect_radio4),np.mean(Seed_aspect_radio5),np.mean(Seed_aspect_radio6)],
                [average_no_zero(White_time1),average_no_zero(White_time2),average_no_zero(White_time3),average_no_zero(White_time4),average_no_zero(White_time5),average_no_zero(White_time6)],
                [np.mean(Linear_MSD1),np.mean(Linear_MSD2),np.mean(Linear_MSD3),np.mean(Linear_MSD4),np.mean(Linear_MSD5),np.mean(Linear_MSD6)],
                ]
                ).T

    columns_name = ["Seed_length(mm)","Seed_Width(mm)","Seed_Area(mm²)","Seed_Perimeter(mm)","Seed_Eccentricity(0-1)","Seed_Roundness(0-1)","Seed_Aspect_Radio","White_Time(hours)","MSD(mm²)"]
    rows_name = ["Region_one","Region_two","Region_three","Region_four","Region_five","Region_six"]
    df_seed_traits.columns= columns_name
    df_seed_traits.index= rows_name
    df_seed_traits.to_csv(os.path.join(Save_dir,"Seed_Traits.csv"))
    ################///////////////////////////// @2  ////////////////######################################  
    df_Frame_Width = pd.DataFrame(np.concatenate([germinate_rectangle_width_1,germinate_rectangle_width_2,germinate_rectangle_width_3,\
                                germinate_rectangle_width_4,germinate_rectangle_width_5,germinate_rectangle_width_6]))
    df_Frame_Width.columns=x
    df_Frame_Width.index=y1
    df_Frame_Width.to_csv(os.path.join(Save_dir,"Frame_Width_Traits.csv"))
    ################///////////////////////////// @3  ////////////////######################################  
    df_Frame_Length = pd.DataFrame(np.concatenate([germinate_rectangle_length_1,germinate_rectangle_length_2,germinate_rectangle_length_3,\
                                germinate_rectangle_length_4,germinate_rectangle_length_5,germinate_rectangle_length_6]))
    df_Frame_Length.columns=x
    df_Frame_Length.index=y1
    df_Frame_Length.to_csv(os.path.join(Save_dir,"Frame_Length_Traits.csv"))
    ################///////////////////////////// @4  ////////////////######################################  
    df_Frame_Aspect_Radio = pd.DataFrame(np.concatenate([germinate_rectangle_aspect_radio_1,germinate_rectangle_aspect_radio_2,germinate_rectangle_aspect_radio_3,\
                                germinate_rectangle_aspect_radio_4,germinate_rectangle_aspect_radio_5,germinate_rectangle_aspect_radio_6]))
    df_Frame_Aspect_Radio.columns=x
    df_Frame_Aspect_Radio.index=y1
    df_Frame_Aspect_Radio.to_csv(os.path.join(Save_dir,"Frame_Aspect_Radio_Traits.csv"))
    ################///////////////////////////// @5  ////////////////######################################  
    df_Frame_Area = pd.DataFrame(np.concatenate([germinate_rectangle_area_1,germinate_rectangle_area_2,germinate_rectangle_area_3,\
                                germinate_rectangle_area_4,germinate_rectangle_area_5,germinate_rectangle_area_6]))
    df_Frame_Area.columns=x
    df_Frame_Area.index=y2
    df_Frame_Area.to_csv(os.path.join(Save_dir,"Frame_Area_Traits.csv"))

    ################///////////////////////////// @6  ////////////////######################################  
    df_Frame_Perimeter = pd.DataFrame(np.concatenate([germinate_rectangle_perimeter_1,germinate_rectangle_perimeter_2,germinate_rectangle_perimeter_3,\
                                germinate_rectangle_perimeter_4,germinate_rectangle_perimeter_5,germinate_rectangle_perimeter_6]))
    df_Frame_Perimeter.columns=x
    df_Frame_Perimeter.index=y1
    df_Frame_Perimeter.to_csv(os.path.join(Save_dir,"Frame_Perimeter_Traits.csv"))
    ################/////////////////////////////@7  ////////////////######################################  
    df_Frame_Area_Growth_Rate  = pd.DataFrame(np.concatenate([germinate_rectangle_speed1,germinate_rectangle_speed2,germinate_rectangle_speed3,\
                                germinate_rectangle_speed4,germinate_rectangle_speed5,germinate_rectangle_speed6]))
    df_Frame_Area_Growth_Rate.columns=x1
    df_Frame_Area_Growth_Rate.index=y2
    df_Frame_Area_Growth_Rate.to_csv(os.path.join(Save_dir,"Frame_Area_Growth_Rate_Traits.csv"))
    ################///////////////////////////// @8  ////////////////######################################  
    df_Growth_Track_speed  = pd.DataFrame(np.concatenate([Linear_speed1,Linear_speed2,Linear_speed3,\
                                Linear_speed4,Linear_speed5,Linear_speed6]))
    df_Growth_Track_speed.columns=x1
    df_Growth_Track_speed.index=y3
    df_Growth_Track_speed.to_csv(os.path.join(Save_dir,"Growth_Track_speed_Traits.csv"))
    ################///////////////////////////// @9  ////////////////######################################  
    df_Growth_Track_Length  = pd.DataFrame(np.concatenate([Linear_distance1,Linear_distance2,Linear_distance3,\
                                Linear_distance4,Linear_distance5,Linear_distance6]))
    df_Growth_Track_Length.columns= x
    df_Growth_Track_Length.index=y1
    df_Growth_Track_Length.to_csv(os.path.join(Save_dir,"Growth_Track_Length.csv"))
    ################///////////////////////////// @10  ////////////////######################################  
    df_Root_Length  = pd.DataFrame(np.concatenate([Root_length1,Root_length2,Root_length3,\
                                Root_length4,Root_length5,Root_length6]))
    df_Root_Length.columns= x
    df_Root_Length.index=y1
    df_Root_Length.to_csv(os.path.join(Save_dir,"Root_Length.csv"))
    ################///////////////////////////// @11  ////////////////######################################  
    df_White_Radio  = pd.DataFrame(np.concatenate([White_radio1,White_radio2,White_radio3,\
                                White_radio4,White_radio5,White_radio6]))
    df_White_Radio.columns= x
    df_White_Radio.index=y1
    df_White_Radio.to_csv(os.path.join(Save_dir,"White_Radio.csv"))
    ################///////////////////////////// @12  ////////////////######################################  
    df_Root_Diameter  = pd.DataFrame(np.concatenate([Root_diameter1,Root_diameter2,Root_diameter3,\
                                Root_diameter4,Root_diametedr5,Root_diameter6]))
    df_Root_Diameter.columns= x
    df_Root_Diameter.index=y1
    df_Root_Diameter.to_csv(os.path.join(Save_dir,"Root_Diameter.csv"))
    ################/////////////////////////////@13  ////////////////######################################  
    df_Root_Area  = pd.DataFrame(np.concatenate([Root_area1,Root_area2,Root_area3,\
                                Root_area4,Root_area5,Root_area6]))
    df_Root_Area.columns= x
    df_Root_Area.index=y2
    df_Root_Area.to_csv(os.path.join(Save_dir,"Root_Area.csv"))
    ################///////////////////////////// @14  ////////////////######################################  
    df_Root_Perimeter  = pd.DataFrame(np.concatenate([Root_permiter1,Root_permiter2,Root_permiter3,\
                                Root_permiter4,Root_permiter5,Root_permiter6]))
    df_Root_Perimeter.columns= x
    df_Root_Perimeter.index=y1
    df_Root_Perimeter.to_csv(os.path.join(Save_dir,"Root_Perimeter.csv")) 

    ################///////////////////////////// @15  ////////////////######################################  
    df_Shoot_Length  = pd.DataFrame(np.concatenate([Shoot_length1,Shoot_length2,Shoot_length3,\
                                Shoot_length4,Shoot_length5,Shoot_length6]))
    df_Shoot_Length.columns= x
    df_Shoot_Length.index=y1
    df_Shoot_Length.to_csv(os.path.join(Save_dir,"Shoot_Length.csv"))
    # ################///////////////////////////// @16  ////////////////######################################  
    df_Shoot_Area  = pd.DataFrame(np.concatenate([Shoot_area1,Shoot_area2,Shoot_area3,\
                                Shoot_area4,Shoot_area5,Shoot_area6]))
    df_Shoot_Area.columns= x
    df_Shoot_Area.index=y2
    df_Shoot_Area.to_csv(os.path.join(Save_dir,"Shoot_Area.csv"))
    ################///////////////////////////// @17  ////////////////######################################  
    df_Shoot_Diameter  = pd.DataFrame(np.concatenate([Shoot_diameter1,Shoot_diameter2,Shoot_diameter3,\
                            Shoot_diameter4,Shoot_diameter5,Shoot_diameter6]))
    df_Shoot_Diameter.columns= x
    df_Shoot_Diameter.index=y1
    df_Shoot_Diameter.to_csv(os.path.join(Save_dir,"Shoot_Diameter.csv"))
    ################///////////////////////////// @18  ////////////////######################################  
    df_Germination_Rate  = pd.DataFrame(np.concatenate([Germination_rate1,Germination_rate2,Germination_rate3,\
                            Germination_rate4,Germination_rate5,Germination_rate6]))
    df_Germination_Rate.columns= x
    df_Germination_Rate.index=y1
    df_Germination_Rate.to_csv(os.path.join(Save_dir,"Germination_Rate.csv"))
    ################///////////////////////////// @19  ////////////////######################################  
    df_Shoot_Perimeter  = pd.DataFrame(np.concatenate([Shoot_permiter1,Shoot_permiter2,Shoot_permiter3,\
                            Shoot_permiter4,Shoot_permiter5,Shoot_permiter6]))
    df_Shoot_Perimeter.columns= x
    df_Shoot_Perimeter.index=y1
    df_Shoot_Perimeter.to_csv(os.path.join(Save_dir,"Shoot_Perimeter.csv"))
    ################///////////////////////////// @20  ////////////////######################################  
    df_Curly_Degree  = pd.DataFrame(np.concatenate([ Curling_degree1, Curling_degree2,Curling_degree3,\
                            Curling_degree4,Curling_degree5,Curling_degree6]))
    df_Curly_Degree.columns= x
    df_Curly_Degree.index=y4
    df_Curly_Degree.to_csv(os.path.join(Save_dir,"Curly_Degree.csv"))
    
print("Finished running, results have been saved to folder")
print(time.strftime('%Y-%m-%d %H:%M:%S',time.localtime(time.time()))) 




  0%|          | 0/100 [00:00<?, ?it/s]

C:\Users\29209\AppData\Local\Temp\ipykernel_27316\698539066.py:28: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  if next_box ==[]:
  0%|          | 0/100 [04:11<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
###################################################Algorithm ends##########################################################
# The code above is a complete script that processes images to extract various traits related to seed germination and growth.
# It includes functions for detecting regions in images, calculating traits such as seed length, width, 

In [ ]:
###################################################Algorithm ends##########################################################
# The code above is a complete script that processes images to extract various traits related to seed germination and growth.
# It includes functions for detecting regions in images, calculating traits such as seed length, width, 

In [ ]:
###################################################Algorithm ends##########################################################
# The code above is a complete script that processes images to extract various traits related to seed germination and growth.
# It includes functions for detecting regions in images, calculating traits such as seed length, width, 